# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
DATA_FOLDER = "/home/vinz/Desktop/ADA/ADA2017-Tutorials/02 - Intro to Pandas/Data" # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average* per year of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

### Strategy

We will import and wrangle the data for each country separately and then combine everything into a single data frame.

As the data files already contain variables with nation wide statistics, we will use these values instead of manually aggregating the data city by city.

As specified on Mattermost, we track for both new cases and deaths the probable, suspected and confirmed values.

In [ ]:
def ebola_csv_iter(folder):
    """Utility function returning an iterator over the csv files in given folder"""
    path = ''.join([DATA_FOLDER, '/ebola/', folder, '/'])
    for file in os.listdir(path):
        if file.endswith(".csv"):
            yield file, ''.join([path, file])

In [ ]:
def associate_to_country(country, df):
    """Adds a country super index to the provided dataframe"""
    country = pd.DataFrame({
        'Country': np.repeat(country, len(df)),
        'Date': df.index
    })
    return country.merge(df, right_index=True, left_on='Date').set_index(['Country', 'Date'])

In [ ]:
def parse_num(x):
    try:
        return np.float32(x)
    except ValueError:
        return np.nan

### Guinea

In [ ]:
dfs = {}
tables_with_duplicates = []
interesting_fields = {
    #'New deaths registered today (confirmed)': 'Death confirmed',
    #'New deaths registered today (probables)': 'Death probable',
    #'New deaths registered today (suspects)': 'Death suspected',
    'Total deaths of suspects': 'Death suspected',
    'Total deaths of probables': 'Death probable',
    'Total deaths of confirmed': 'Death confirmed',
    'New cases of confirmed': 'New cases confirmed',
    'New cases of probables': 'New cases probable', 
    'New cases of suspects': 'New cases suspected'
}
for file, path in ebola_csv_iter('guinea_data'):
    df = pd.read_csv(path,
                     parse_dates=['Date'],
                     usecols=['Date', 'Description', 'Totals'],
                     converters={'Totals': parse_num})
    df.rename(columns={'Description': 'Variable', 'Totals': 'National'}, inplace=True)
    # Check for duplicates
    df = df[df.Variable.isin(interesting_fields)]
    if len(df.Variable.unique()) != len(df.Variable):
        tables_with_duplicates.append(file)
    dfs[file] = df

In [ ]:
tables_with_duplicates

As there are no duplicates, we can pivot all the dataframes to have a timeseries dataframe:

In [ ]:
dfs = [df.pivot(index='Date', columns='Variable', values='National') for df in dfs.values()]

In [ ]:
guinea = pd.concat(dfs).sort_index() # Create the data series: index=date
guinea = associate_to_country('Guinea', guinea).rename(columns=interesting_fields)

We started by using the fields:

 - `New deaths registered today (confirmed)`
 - `New deaths registered today (probables)`
 - `New deaths registered today (suspects)`
 
but they were empty for all days except one. Therefore we decided to use the cumulated values and perform the subtraction to obtain daily counts:

In [ ]:
deaths = guinea[['Death confirmed', 'Death probable', 'Death suspected']]
cleaned = deaths - deaths.shift(1) # row_i - row_{i - 1}
guinea.loc[:, ['Death confirmed', 'Death probable', 'Death suspected']] = cleaned

This operation relies on a very important assumption. 
We repeat the same computation on the data concerning Liberia and Sierra Leone.
We assume that, given that the counts are cumulative, if we are missing a value for day $j$ and we have a value for day $i > j$, the value for day $j$ is counted in day $i$. **In other words, even if data for some days is missing, the totals are correct.**

In [ ]:
guinea.head(n=10)

### Liberia

In [ ]:
dfs = {}
tables_with_duplicates = []
interesting_fields = {
    'Total death/s in confirmed cases': 'Death confirmed',
    'Total death/s in probable cases': 'Death probable',
    'Total death/s in suspected cases': 'Death suspected',
    'New case/s (confirmed)': 'New cases confirmed',
    'New Case/s (Probable)': 'New cases probable',
    'New Case/s (Suspected)': 'New cases suspected'
}
for file, path in ebola_csv_iter('liberia_data'):
    df = pd.read_csv(path,
                     parse_dates=['Date'],
                     usecols=['Date', 'Variable', 'National'],
                     converters={'National': parse_num})
    df = df[df.Variable.isin(interesting_fields)]
    # Check for duplicates
    if len(df.Variable.unique()) != len(df.Variable):
        tables_with_duplicates.append(file)
    dfs[file] = df

We remark that only a data file contains duplicate variables for the same day. Before dropping the duplicates, we look at the duplicated lines:

In [ ]:
tables_with_duplicates

In [ ]:
with_duplicates = dfs[tables_with_duplicates[0]]
with_duplicates[with_duplicates.duplicated(keep=False, subset='Variable')]

7 Variables are repeated twice.

In [ ]:
first = with_duplicates[with_duplicates.duplicated(keep='last', subset='Variable')][['Variable', 'National']]
second = with_duplicates[with_duplicates.duplicated(keep='first', subset='Variable')][['Variable', 'National']]
first = first.set_index('Variable')
second = second.set_index('Variable')

In [ ]:
second - first

We decided to keep the bigger values as the difference is not very relevant:

In [ ]:
with_duplicates.drop_duplicates(subset='Variable', keep='last', inplace=True)

In [ ]:
dfs = [df.pivot(index='Date', columns='Variable', values='National') for df in dfs.values()]

In [ ]:
liberia = pd.concat(dfs).sort_index()
liberia = associate_to_country('Liberia', liberia).rename(columns=interesting_fields)

In [ ]:
liberia.head(n=10)

However in this dataframe, the three deaths columns are *cumulative*. We can therefore subtract the value from the previous line.

In [ ]:
deaths = liberia[['Death confirmed', 'Death probable', 'Death suspected']]
cleaned = deaths - deaths.shift(1) # row_i - row_{i - 1}
liberia.loc[:, ['Death confirmed', 'Death probable', 'Death suspected']] = cleaned

In [ ]:
to_scale = liberia[-6:][['New cases probable', 'New cases suspected', 'New cases confirmed']]

In [ ]:
liberia.loc[-6:, ['New cases probable', 'New cases suspected', 'New cases confirmed']] = to_scale - to_scale.shift(1)

### Sierra Leone

In [ ]:
dfs = {}
tables_with_duplicates = []
interesting_fields = {
    'death_confirmed': 'Death confirmed',
    'death_probable': 'Death probable',
    'death_suspected': 'Death suspected',
    'new_confirmed': 'New cases confirmed',
    'new_probable': 'New cases probable',
    'new_suspected': 'New cases suspected'
}

for file, path in ebola_csv_iter('sl_data'):
    df = pd.read_csv(path,
                     parse_dates=['date'],
                     usecols=['date', 'variable', 'National'], 
                     converters={'National': parse_num})
    df.rename(columns={'date': 'Date', 'variable': 'Variable'}, inplace=True)
    df = df[df.Variable.isin(interesting_fields)]
    if len(df.Variable.unique()) != len(df.Variable):
        tables_with_duplicates.append(file)
    dfs[file] = df

In [ ]:
tables_with_duplicates

In [ ]:
dfs = [df.pivot(index='Date', columns='Variable', values='National') for df in dfs.values()]

In [ ]:
sl = pd.concat(dfs).sort_index()
sl = associate_to_country('Sierra Leone', sl).rename(columns=interesting_fields)

In [ ]:
sl.head(n=10)

The fields concerning the number of deaths are cumulative. We apply the same treatment as in the previous cases:

In [ ]:
deaths = sl[['Death confirmed', 'Death probable', 'Death suspected']]
cleaned = deaths - deaths.shift(1) # row_i - row_{i - 1}
sl.loc[:, ['Death confirmed', 'Death probable', 'Death suspected']] = cleaned

In [ ]:
del dfs
del tables_with_duplicates

### Combining the data

In [ ]:
# Concatenate all dataframes
df = pd.concat([guinea, liberia, sl])

### Computing means

In [ ]:
def month_average(group):
    # Only divide by the number of days counted in the set.
    registered_days = group.index.get_level_values(1).max().day
    return group.sum() / registered_days

In [ ]:
import calendar
# Compute mean per month per country. 
results = df.groupby(lambda row: (row[0], row[1].month))\
            .agg(month_average)

# Give computation result a nice index
results.index = pd.MultiIndex.from_tuples(results.index, names=['Country', 'Month'])\
                  .map(lambda i: (i[0], calendar.month_name[i[1]]))

In [ ]:
results

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here